In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import torch
import pickle
import os

In [2]:
class SimpleBBCEmbeddingGenerator:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)
        self.model_name = model_name
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        if torch.cuda.is_available():
            self.model = self.model.to(self.device)

    def load_bbc_dataset(self, csv_path='bbc_encoded.csv', text_column='text'):
        df = pd.read_csv(csv_path)
        df[text_column] = df[text_column].astype(str)
        df = df.dropna(subset=[text_column])
        df = df[df[text_column].str.strip() != '']
        return df, text_column

    def generate_embeddings(self, texts, batch_size=32, show_progress=True):
        if hasattr(texts, 'tolist'):
            texts = texts.tolist()

        embeddings = self.model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=show_progress,
            convert_to_numpy=True,
            device=self.device
        )
        return embeddings

    def save_embeddings(self, embeddings):
        embeddings_path = f"bbc_sbert_embeddings.npy"
        np.save(embeddings_path, embeddings)
        return embeddings_path

In [3]:
def main():
    CSV_FILE = 'bbc_encoded.csv'
    TEXT_COLUMN = 'text'
    MODEL_NAME = 'all-MiniLM-L6-v2'
    BATCH_SIZE = 32
    SAVE_PREFIX = 'bbc_sbert'

    try:
        generator = SimpleBBCEmbeddingGenerator(model_name=MODEL_NAME)

        df, text_col = generator.load_bbc_dataset(CSV_FILE, TEXT_COLUMN)

        texts = df[text_col].tolist()

        embeddings = generator.generate_embeddings(
            texts,
            batch_size=BATCH_SIZE,
            show_progress=True
        )

        embeddings_path = generator.save_embeddings(
            embeddings,
            SAVE_PREFIX
        )

        return embeddings

    except Exception as e:
        print(f"Error in pipeline: {e}")
        raise


if __name__ == "__main__":
    embeddings = main()

Loading SBERT model: all-MiniLM-L6-v2


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Using device: cpu
Loading dataset from: bbc_encoded.csv
Dataset loaded successfully!
Shape: (2127, 2)
Columns: ['text', 'label_encoded']
After cleaning: 2127 rows

Processing 2127 texts...
Generating embeddings for 2127 texts...
Model: all-MiniLM-L6-v2
Batch size: 32


Batches:   0%|          | 0/67 [00:00<?, ?it/s]

Generated embeddings shape: (2127, 384)
Embedding dimension: 384

Saving results...
✓ Saved embeddings to: bbc_sbert_embeddings.npy

SBERT EMBEDDING GENERATION COMPLETED!
Embeddings saved to: bbc_sbert_embeddings.npy
Shape: (2127, 384)
Model used: all-MiniLM-L6-v2
